In [0]:
SELECT * FROM gizmobox.bronze.v_customers where customer_id IS NOT NULL
ORDER BY customer_id

In [0]:
SELECT DISTINCT * FROM gizmobox.bronze.v_customers where customer_id IS NOT NULL
ORDER BY customer_id

In [0]:
SELECT customer_id, MAX(created_timestamp)
FROM gizmobox.bronze.v_customers 
WHERE customer_id IS NOT NULL 
GROUP BY customer_id
ORDER BY customer_id

In [0]:
CREATE OR REPLACE TEMPORARY VIEW v_customers_distinct AS
SELECT DISTINCT * FROM gizmobox.bronze.v_customers 
WHERE customer_id IS NOT NULL ORDER BY customer_id

In [0]:
SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
FROM v_customers_distinct GROUP BY customer_id

In [0]:
WITH cte_max AS (
  SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
  FROM v_customers_distinct GROUP BY customer_id
)
SELECT CAST(t.created_timestamp AS TIMESTAMP) as created_timestamp,
t.customer_id, t.customer_name, CAST(t.date_of_birth AS DATE) AS date_of_birth,
t.email, CAST(t.member_since AS DATE) as member_since, t.telephone
FROM v_customers_distinct t
JOIN cte_max m ON t.customer_id = m.customer_id AND t.created_timestamp = m.max_created_timestamp

In [0]:
CREATE TABLE gizmobox.silver.customers
AS
WITH cte_max AS (
  SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
  FROM v_customers_distinct GROUP BY customer_id
)
SELECT CAST(t.created_timestamp AS TIMESTAMP) as created_timestamp,
t.customer_id, t.customer_name, CAST(t.date_of_birth AS DATE) AS date_of_birth,
t.email, CAST(t.member_since AS DATE) as member_since, t.telephone
FROM v_customers_distinct t
JOIN cte_max m ON t.customer_id = m.customer_id AND t.created_timestamp = m.max_created_timestamp

In [0]:
SELECT * FROM gizmobox.silver.customers

In [0]:
SELECT payment_id, payment_timestamp, payment_status, payment_method FROM gizmobox.bronze.payments

In [0]:
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  CASE payment_status 
    WHEN 1 THEN 'Success' 
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'FAILED'
    END AS payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
CREATE TABLE gizmobox.silver.payments
AS
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  CASE payment_status 
    WHEN 1 THEN 'Success' 
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'FAILED'
    END AS payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
SELECT * FROM gizmobox.silver.payments

In [0]:
SELECT 
  refund_id, 
  payment_id,
  refund_amount,
  refund_reason
FROM hive_metastore.bronze.refunds

In [0]:
SELECT 
  refund_id, 
  payment_id,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
SELECT 
  refund_id, 
  payment_id,
  CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
  date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
CREATE SCHEMA hive_metastore.silver

In [0]:
CREATE TABLE hive_metastore.silver.refunds AS
SELECT 
  refund_id, 
  payment_id,
  CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
  date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
SELECT * FROM hive_metastore.silver.refunds

In [0]:
DESC EXTENDED hive_metastore.silver.refunds

In [0]:
SELECT * FROM gizmobox.bronze.v_memberships

In [0]:
SELECT 
regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
content as membership_card
FROM gizmobox.bronze.v_memberships

In [0]:
CREATE TABLE gizmobox.silver.membership_cards AS
SELECT 
regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
content as membership_card
FROM gizmobox.bronze.v_memberships

In [0]:
SELECT * FROM gizmobox.silver.membership_cards

In [0]:
SELECT 
  customer_id,
  address_type,
  address_line_1,
  city,
  state,
  postcode
FROM gizmobox.bronze.v_addresses

In [0]:
SELECT *
FROM (
  SELECT 
  customer_id,
  address_type,
  address_line_1,
  city,
  state,
  postcode
FROM gizmobox.bronze.v_addresses)
PIVOT (MAX(address_line_1) AS address_line_1,
       MAX(city) AS city,
       MAX(state) AS state,
       MAX(postcode) AS postcode
       FOR address_type IN ('shipping', 'billing')
)

In [0]:
CREATE TABLE gizmobox.silver.addresses AS
SELECT *
FROM (
  SELECT 
  customer_id,
  address_type,
  address_line_1,
  city,
  state,
  postcode
FROM gizmobox.bronze.v_addresses)
PIVOT (MAX(address_line_1) AS address_line_1,
       MAX(city) AS city,
       MAX(state) AS state,
       MAX(postcode) AS postcode
       FOR address_type IN ('shipping', 'billing')
)

In [0]:
SELECT * FROM gizmobox.silver.addresses

In [0]:
SELECT * 
FROM gizmobox.bronze.v_orders

In [0]:
SELECT value:order_id AS order_id, value
FROM gizmobox.bronze.v_orders

In [0]:
SELECT 
  value:items[0] AS item_1, 
  value:items[1] AS item_2, value
FROM gizmobox.bronze.v_orders

In [0]:
SELECT 
  value:items[0].item_id AS item_1, 
  value:items[1] AS item_2, value
FROM gizmobox.bronze.v_orders

In [0]:
SELECT 
  value:items[0].item_id::INTEGER AS item_1, 
  value:items[1] AS item_2, value
FROM gizmobox.bronze.v_orders

In [0]:
CREATE OR REPLACE TEMP VIEW tv_orders_fixed AS
SELECT 
  value,
  regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM gizmobox.bronze.v_orders

In [0]:
SELECT fixed_value
FROM tv_orders_fixed

In [0]:
SELECT schema_of_json(fixed_value) AS schema, fixed_value
FROM tv_orders_fixed LIMIT 1;

In [0]:
SELECT from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value, fixed_value
FROM tv_orders_fixed LIMIT 1;

In [0]:
DROP TABLE IF EXISTS gizmobox.silver.orders_json;
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
AS
SELECT from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') AS json_value
FROM tv_orders_fixed

In [0]:
SELECT * FROM gizmobox.silver.orders_json

In [0]:
DESCRIBE EXTENDED gizmobox.silver.orders_json;

In [0]:
SELECT * FROM gizmobox.silver.orders_json

In [0]:
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  array_distinct(json_value.items) AS items
FROM gizmobox.silver.orders_json

In [0]:
CREATE OR REPLACE TEMP VIEW tv_orders_exploded AS
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) AS item
FROM gizmobox.silver.orders_json

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders AS
SELECT 
  order_id, 
  order_status,
  total_amount,
  transaction_timestamp,
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
FROM tv_orders_exploded

In [0]:
SELECT * FROM gizmobox.silver.orders

In [0]:
SELECT * FROM gizmobox.silver.customers

In [0]:
SELECT * FROM gizmobox.silver.addresses

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.gold.customer_address
SELECT 
  c.customer_id,
  c.customer_name,
  c.email,
  c.date_of_birth,
  c.member_since,
  c.telephone,
  a.shipping_address_line_1,
  a.shipping_city,
  a.shipping_state,
  a.shipping_postcode,
  a.billing_address_line_1,
  a.billing_city,
  a.billing_state,
  a.billing_postcode
FROM gizmobox.silver.customers c
JOIN gizmobox.silver.addresses a
ON c.customer_id = a.customer_id

In [0]:
SELECT * FROM gizmobox.gold.customer_address